# Image - Mask - Annotation Matching

In [1]:
%cd ../..

/home/bhkuser/bhklab/katy/recist-vs-reality


In [2]:
import pandas as pd
from pathlib import Path
from damply import dirs
from readii.io.loaders import loadImageDatasetConfig

/home/bhkuser/bhklab/katy/recist-vs-reality/.pixi/envs/default/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = "CPTAC-CCRCC"

config = loadImageDatasetConfig(dataset, dirs.CONFIG)

procdata_dir = dirs.PROCDATA / f"{config['DATA_SOURCE']}_{config['DATASET_NAME']}"

In [4]:
annotations_file = procdata_dir / "annotations" / f"{dataset}_graphic_data.csv"
ann_graphic_data = pd.read_csv(annotations_file)

annotation_series = pd.Index(ann_graphic_data['Series Instance UID'])
annotation_series.unique()

Index(['1.3.6.1.4.1.14519.5.2.1.2857.3304.244060230648202689800516272542',
       '1.3.6.1.4.1.14519.5.2.1.3320.3273.198272441931866966244329638329',
       '1.3.6.1.4.1.14519.5.2.1.7085.2626.917594956182057044787724210431',
       '1.3.6.1.4.1.14519.5.2.1.3320.3273.100156103408987714172612199049',
       '1.3.6.1.4.1.14519.5.2.1.7085.2626.241290796655397384297058111354',
       '1.3.6.1.4.1.14519.5.2.1.7085.2626.294375358125602155777716850534',
       '1.3.6.1.4.1.14519.5.2.1.3320.3273.183236381742584749034608925853',
       '1.3.6.1.4.1.14519.5.2.1.3320.3273.141587574827399640433864402196',
       '1.3.6.1.4.1.14519.5.2.1.3320.3273.282697041835985489855755847747',
       '1.3.6.1.4.1.14519.5.2.1.3320.3273.167452856540705803530972968910',
       '1.3.6.1.4.1.14519.5.2.1.3320.3273.261097023551283782286933309661',
       '1.3.6.1.4.1.14519.5.2.1.2932.1975.504686912389793508180110599728',
       '1.3.6.1.4.1.14519.5.2.1.3320.3273.191150767462242610143736498429',
       '1.3.6.1.4.1.14519

In [5]:
mit_index_file = procdata_dir / "images" / f"mit_{dataset}" / f"mit_{dataset}_index-simple.csv"
mit_index = pd.read_csv(mit_index_file)

mit_series = pd.Index(mit_index['SeriesInstanceUID'])

overlapping_series = annotation_series.intersection(mit_series)

In [6]:
image_rows = mit_index[mit_index['SeriesInstanceUID'].isin(overlapping_series)]
mask_rows = mit_index[mit_index['ReferencedSeriesUID'].isin(overlapping_series)]

In [7]:
len(annotation_series.unique())

59

In [8]:
len(annotation_series)/2

102.0

# Get segmentation metadata for the overlap subset

In [9]:
# Load subset file
overlap_metadata = pd.read_csv(dirs.METADATA / "RvR_patient_id_overlap_seg_ann.csv")

dataset_overlap_metadata = overlap_metadata[overlap_metadata['Collection'] == dataset]


In [10]:
tumor_metadata_file_name = f"{dataset}_{config['MIT']['MODALITIES']['mask']}_tumor_summary.csv"

dataset_tumor_metadata = pd.read_csv(dirs.RAWDATA / f"{config['DATA_SOURCE']}_{config['DATASET_NAME']}" / "metadata" / tumor_metadata_file_name)

In [11]:
overlap_tumor_metadata = dataset_tumor_metadata[dataset_tumor_metadata['PatientID'].isin(dataset_overlap_metadata['SubjectID'])]
overlap_tumor_metadata.reset_index(drop=True, inplace=True)

In [12]:
overlap_tumor_metadata

,PatientID,StudyInstanceUID,SeriesInstanceUID,Modality,ReferencedSeriesInstanceUID,OriginalNumberOfROIs,ExtractedNumberOfROIs,OriginalROINames,ExtractableROINames,ReferencedModality
0,C3N-00953,1.3.6.1.4.1.14519.5.2.1.7085.2626.192997540292...,1.2.826.0.1.534147.667.2747872357.202342361524809,RTSTRUCT,1.3.6.1.4.1.14519.5.2.1.7085.2626.126066463046...,1,1,['LT KIDNEY - 1 -'],['LT KIDNEY - 1 -'],CT
1,C3N-02015,1.3.6.1.4.1.14519.5.2.1.7085.2626.265971496207...,1.2.826.0.1.534147.756.812677238.2023222546335...,RTSTRUCT,1.3.6.1.4.1.14519.5.2.1.7085.2626.241290796655...,1,1,['LT KIDNEY - 1 -'],['LT KIDNEY - 1 -'],CT
2,C3N-02015,1.3.6.1.4.1.14519.5.2.1.7085.2626.265971496207...,1.2.826.0.1.534147.756.812677238.2023222547548...,RTSTRUCT,1.3.6.1.4.1.14519.5.2.1.7085.2626.241290796655...,1,1,['LT KIDNEY - 1'],['LT KIDNEY - 1'],CT
3,C3N-02015,1.3.6.1.4.1.14519.5.2.1.7085.2626.265971496207...,1.2.826.0.1.534147.667.2747872357.2023419637733,RTSTRUCT,1.3.6.1.4.1.14519.5.2.1.7085.2626.203408914239...,1,1,['LT KIDNEY - 1 -'],['LT KIDNEY - 1 -'],CT
4,C3N-00953,1.3.6.1.4.1.14519.5.2.1.7085.2626.192997540292...,1.2.826.0.1.534147.667.2747872357.202342361517...,RTSTRUCT,1.3.6.1.4.1.14519.5.2.1.7085.2626.126066463046...,1,1,['LT KIDNEY - 1'],['LT KIDNEY - 1'],CT
...,...,...,...,...,...,...,...,...,...,...
187,C3N-00494,1.3.6.1.4.1.14519.5.2.1.3320.3273.254343482420...,1.2.826.0.1.534147.667.2747872357.202341910442...,RTSTRUCT,1.3.6.1.4.1.14519.5.2.1.3320.3273.296322667616...,1,1,['BLADDER - 1'],['BLADDER - 1'],CT
188,C3N-00492,1.3.6.1.4.1.14519.5.2.1.3320.3273.241341825292...,1.2.826.0.1.534147.667.2747872357.202341961253...,RTSTRUCT,1.3.6.1.4.1.14519.5.2.1.3320.3273.130827027987...,1,1,['RT KIDNEY - 1'],['RT KIDNEY - 1'],CT
189,C3N-01178,1.3.6.1.4.1.14519.5.2.1.3320.3273.136103785758...,1.2.826.0.1.534147.667.2747872357.202342363817...,RTSTRUCT,1.3.6.1.4.1.14519.5.2.1.3320.3273.167452856540...,1,1,['RT KIDNEY - 1 -'],['RT KIDNEY - 1 -'],CT
190,C3N-00492,1.3.6.1.4.1.14519.5.2.1.3320.3273.241341825292...,1.2.826.0.1.534147.667.2747872357.202341971022...,RTSTRUCT,1.3.6.1.4.1.14519.5.2.1.3320.3273.305579371316...,1,1,['RT KIDNEY - 1'],['RT KIDNEY - 1'],CT
